In [ ]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import re
import pandas as pd
import numpy as np

In [ ]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t
def get_drugbox(s):
    beg = (s.rfind('{{Drugbox'))
    end  =(s.rfind('\n}}'))
    if( end == -1):
        end = end =(s.rfind('}}\n'))
    if( end == -1):
        end = end =(s.rfind('}}\n<!--'))
    if( end == -1):
        end = end =(s.rfind('}}\n=='))
    if( end == -1):
        end = end =(s.rfind('}}\n\d'))
    s = s[beg: end+2]
    return s
def get_med_cond(s):
    beg = (s.rfind('{{Infobox medical condition'))
    end  =(s.rfind('\n}}'))
    if( end == -1):
        end = end =(s.rfind('}}\n'))
    if( end == -1):
        end = end =(s.rfind('}}\n<!--'))
    if( end == -1):
        end = end =(s.rfind('}}\n=='))
    if( end == -1):
        end = end =(s.rfind('}}\n\d'))
    s = s[beg: end+2]
    return s
def get_medical(s):
    beg = (s.rfind('{{Medical resources'))
    end  =(s.rfind('\n}}'))
    s = s[beg: end+3]
    return s
def find_unii(s):
    s = re.findall(r'UNII\s*?=\s?.*',s)
    if(len(s)>0):   
        s = s[0]
        equal = s.rfind('=')
        #if there is a space after the equal remove it
        if(s[equal+1]==' '):
            s = s[equal+2:]
        else: 
            s = s[equal+1:]
    else:
        s = ''
    return s
def find_icd10(s):
    s = s.replace('|','')
    s = s.replace('{{ICD10',"")
    icd10 = re.findall('\w{1}\d{2,6}',s)
    return icd10

def find_medication(s):
    s = re.findall(r'medication\s*?=.*<',s)
    if(len(s)== 0 ):
        s = []
    else:
        s = s[0]
        s = s[:-1]
        s = s[s.find('=')+2:]
        s = s.replace('[[','')
        s = s.replace(']]','')
        s = s.split(',')
        s
    return s
def pad_array(arr,length):
    for i in range(0,length-len(arr)):
        arr.append(np.NaN)
    return arr
def find_max_medications(df):
    return max([len(x) for x in df[df.medications.map(len) > 0].medications.values])
def get_row_with_medications(df):
    df =  df[df.medications.map(len) > 0]
    df.reset_index(drop=True,inplace=True)
    return df
def set_meds_columns(df,max_col):
    current_meds = df.medications.values
    padded_meds = [pad_array(el,max_col) for el in current_meds]
    cols = [f"medication_{i}" for i in range(max_col)]
    col_df = pd.DataFrame(padded_meds, columns=cols)
    final = pd.concat([df,col_df], axis=1)
    return final
def clean_row(test):
    for i in range (len(test)):
        el = test[i]
        end = el.rfind('<')
        if end !=-1:
            new_el = el[:end]
            test[i]=new_el
        if i>0:
            test[i]= test[i][1:]
    return test

In [ ]:
totalCount = 0
title = None
icd9=[]
icd10=[]
#-----------arrays for drugs_df----------------DO NOT CHANGE NAMES
drugs_name = []
drugs_id = []
drugs_unii = []
#----arrays for diseases_df---------------------DO NOT CHANGE NAMES
diseases_name = []
diseases_id = []
diseases_icd9 = []
diseases_icd10 = []
diseases_meds = []
#-------------------------------------------
start_time = time.time()
for event, elem in etree.iterparse('enwiki-latest-pages-articles.xml', events=('start', 'end')):
    tname = strip_tag_name(elem.tag)

    if event == 'start':
        if tname == 'page':
            title = ''
            id = -1
            redirect = ''
            inrevision = False
            ns = 0
        elif tname == 'revision':
            # Do not pick up on revision id's
            inrevision = True
    else:
        if tname == 'title':
            title = elem.text
        elif tname == 'id' and not inrevision:
            id = int(elem.text)
        elif tname == 'redirect':
            redirect = elem.attrib['title']
        elif tname == 'ns':
            ns = int(elem.text)
        elif tname == 'page':
            totalCount += 1
        elif(tname =='text'):
            cont = elem.text
            #check if cont exists
            if(type(cont)==type('alex')):
                #check if the drugbox template exists 
                if (cont.find('Drugbox')>-1 ):
                    #get the drugbox template
                    result = get_drugbox(cont)
                    #get the CAS number
                    unii = find_unii(result)
                    #drugsWriter.writerow([id, title, cas])
                    drugs_name.append(title)
                    drugs_id.append(id)
                    drugs_unii.append(unii)
                    print(f'{title} {unii}')


elem.clear()
elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

In [ ]:
diseases_with_meds = get_row_with_medications(diseases_df)
diseases_with_meds.medications.apply(clean_row)
max_meds = find_max_medications(diseases_with_meds)
test = set_meds_columns(diseases_with_meds, max_meds)
test